<a href="https://colab.research.google.com/github/LeoDinga/DL_Project/blob/main/Project_DL_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/THETIS-dataset/dataset.git
!mv dataset data

Cloning into 'dataset'...
remote: Enumerating objects: 8471, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 8471 (delta 0), reused 5 (delta 0), pack-reused 8465 (from 1)
Receiving objects: 100% (8471/8471), 12.70 GiB | 17.06 MiB/s, done.
Resolving deltas: 100% (54/54), done.
Updating files: 100% (8379/8379), done.


In [2]:
import os
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as T
import torchvision.models as models
from PIL import Image
import numpy as np
from sklearn.metrics import accuracy_score
import shutil
import random

In [3]:
#shows the videos in each folder
data_path = 'data'

# Print out the directory structure
for root, dirs, files in os.walk(data_path):
    print(f"Root: {root}")
    print(f"Dirs: {dirs}")
    print(f"Files: {files}")
    print("-" * 40)

Root: data
Dirs: ['VIDEO_Mask', '.git', 'papers', 'VIDEO_Skelet3D', 'VIDEO_Skelet2D', 'VIDEO_Depth', 'VIDEO_RGB']
Files: ['README.md']
----------------------------------------
Root: data/VIDEO_Mask
Dirs: ['forehand_flat', 'backhand_slice', 'backhand', 'backhand2hands', 'flat_service', 'forehand_openstands', 'forehand_volley', 'smash', 'backhand_volley', 'slice_service', 'kick_service', 'forehand_slice']
Files: []
----------------------------------------
Root: data/VIDEO_Mask/forehand_flat
Dirs: []
Files: ['p21_foreflat_mask_s2.avi', 'p12_foreflat_mask_s1.avi', 'p50_foreflat_mask_s2.avi', 'p31_foreflat_mask_s2.avi', 'p39_foreflat_mask_s2.avi', 'p45_foreflat_mask_s2.avi', 'p32_foreflat_mask_s1.avi', 'p42_foreflat_mask_s3.avi', 'p21_foreflat_mask_s1.avi', 'p3_foreflat_mask_s2.avi', 'p32_foreflat_mask_s2.avi', 'p4_foreflat_mask_s3.avi', 'p15_foreflat_mask_s3.avi', 'p5_foreflat_mask_s2.avi', 'p33_foreflat_mask_s1.avi', 'p46_foreflat_mask_s3.avi', 'p14_foreflat_mask_s1.avi', 'p36_foreflat_ma

#LEO


In [4]:
def convert_video_to_npy(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Resize frames if needed (e.g., 224x224)
        frame = cv2.resize(frame, (224, 224))
        frames.append(frame)

    cap.release()

    # Convert list of frames into a numpy array
    frames_array = np.array(frames)
    return frames_array

def process_videos_to_npy(video_dir, output_dir):
    for video_filename in os.listdir(video_dir):
        if video_filename.endswith(".avi"):  # Process only .avi files
            video_path = os.path.join(video_dir, video_filename)
            video_name = os.path.splitext(video_filename)[0]

            # Convert the video to numpy array
            frames_array = convert_video_to_npy(video_path)

            # Save the numpy array to a .npy file
            output_filename = os.path.join(output_dir, f"{video_name}.npy")
            np.save(output_filename, frames_array)
            print(f"Saved {video_filename} as {output_filename}")

# Example usage:
videos_dir = 'data/VIDEO_Skelet3D'  # Directory with .avi videos
npy_dir = 'data/npy_videos/'  # Directory to save .npy files
os.makedirs(npy_dir, exist_ok=True)

process_videos_to_npy(videos_dir, npy_dir)

In [5]:
def extract_frames_from_npy(npy_file, num_frames=120):
    # Load the numpy array (which contains all frames of the video)
    frames = np.load(npy_file)

    # Calculate indices to extract 120 evenly spaced frames
    total_frames = frames.shape[0]
    frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)

    # Extract the frames
    extracted_frames = frames[frame_indices]

    return extracted_frames

def process_npy_videos_to_frames(npy_dir, output_dir, num_frames=120):
    for npy_filename in os.listdir(npy_dir):
        if npy_filename.endswith(".npy"):  # Process only .npy files
            npy_file = os.path.join(npy_dir, npy_filename)
            video_name = os.path.splitext(npy_filename)[0]

            # Extract 120 frames from the .npy file
            extracted_frames = extract_frames_from_npy(npy_file, num_frames)

            # Save the extracted frames as a new .npy file
            output_filename = os.path.join(output_dir, f"{video_name}_120frames.npy")
            np.save(output_filename, extracted_frames)
            print(f"Saved 120 frames from {npy_filename} as {output_filename}")

# Example usage:
npy_dir = 'data/npy_videos/'  # Directory with .npy files
output_dir = 'data/120_frames/'  # Directory to save 120 frames
os.makedirs(output_dir, exist_ok=True)

process_npy_videos_to_frames(npy_dir, output_dir)

In [6]:
def split_by_player(video_dir, output_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    # Step 1: Get a list of all players
    players = set()  # Set to store unique players (assuming folder names represent players)

    for video_filename in os.listdir(video_dir):
        if video_filename.endswith(".npy"):  # Only process .npy files
            player_name = video_filename.split('_')[0]  # Assuming player info is in the filename
            players.add(player_name)

    # Step 2: Shuffle the list of players
    players = list(players)
    random.shuffle(players)

    # Step 3: Split players into train/val/test sets
    num_players = len(players)
    train_size = int(train_ratio * num_players)
    val_size = int(val_ratio * num_players)

    train_players = players[:train_size]
    val_players = players[train_size:train_size + val_size]
    test_players = players[train_size + val_size:]

    # Step 4: Create directories for train/val/test sets
    train_dir = os.path.join(output_dir, 'train')
    val_dir = os.path.join(output_dir, 'val')
    test_dir = os.path.join(output_dir, 'test')

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Step 5: Move videos to their respective directories
    for video_filename in os.listdir(video_dir):
        if video_filename.endswith(".npy"):
            player_name = video_filename.split('_')[0]  # Get the player name
            src_path = os.path.join(video_dir, video_filename)

            if player_name in train_players:
                dst_dir = train_dir
            elif player_name in val_players:
                dst_dir = val_dir
            else:
                dst_dir = test_dir

            # Move the file to the appropriate directory
            shutil.copy(src_path, os.path.join(dst_dir, video_filename))
            print(f"Moved {video_filename} to {dst_dir}")

# Example usage:
video_dir = 'data/npy_videos/'  # Directory containing all the .npy files
output_dir = 'data/splits/'  # Directory where train/val/test splits will be saved
os.makedirs(output_dir, exist_ok=True)

split_by_player(video_dir, output_dir)

In [26]:
!git clone https://github.com/yysijie/st-gcn.git
!cd stgcn

fatal: destination path 'st-gcn' already exists and is not an empty directory.
/bin/bash: line 1: cd: stgcn: No such file or directory


In [19]:
ls st-gcn/models/pose/coco

pose_deploy_linevec.prototxt*


In [21]:
!grep -r "class Model" st-gcn/


st-gcn/net/st_gcn.py:class Model(nn.Module):
st-gcn/net/st_gcn_twostream.py:class Model(nn.Module):


In [13]:
!python main.py --config config/stgcn_kinetics.yaml

python3: can't open file '/content/main.py': [Errno 2] No such file or directory


In [7]:
!pip install scipy scikit-learn

In [25]:
!pip install stgcn

ERROR: Could not find a version that satisfies the requirement stgcn (from versions: none)
ERROR: No matching distribution found for stgcn


In [27]:
import torch
from stgcn.net.st_gcn import Model  # Assuming stgcn.py is available from the implementation
from torch import nn

# Load a pre-trained ST-GCN model (Kinetics-Pretrained)
def load_stgcn_model(pretrained=True):
    # Assuming you're using a pre-trained model from Kinetics
    model = Model(num_class=60, num_point=18, in_channels=3, graph_size=18)  # Adjust as per your dataset
    if pretrained:
        model.load_state_dict(torch.load('path_to_pretrained_kinetics_model.pth'))
    model.eval()  # Set the model to evaluation mode
    return model

# Example usage:
stgcn_model = load_stgcn_model(pretrained=True)

ModuleNotFoundError: No module named 'stgcn'

In [ ]:

class KeypointDataset(Dataset):
    def __init__(self, npy_dir, split='train'):
        self.npy_dir = npy_dir
        self.split = split
        self.files = [f for f in os.listdir(npy_dir) if f.endswith('.npy')]

        # If you have a specific player split, apply that here (e.g., train/val/test)
        if self.split == 'train':
            # Load the train files
            self.files = [f for f in self.files if 'player1' in f or 'player2' in f]  # Example player-based split
        elif self.split == 'val':
            # Load the validation files
            self.files = [f for f in self.files if 'player3' in f or 'player4' in f]  # Example player-based split
        else:
            # Load the test files
            self.files = [f for f in self.files if 'player5' in f or 'player6' in f]  # Example player-based split

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        video_filename = self.files[idx]
        video_path = os.path.join(self.npy_dir, video_filename)

        # Load the video keypoints data
        frames = np.load(video_path)  # Shape (T, N, 3) => (120, num_keypoints, 3)

        # Normalize or preprocess data if needed
        frames = torch.tensor(frames, dtype=torch.float32)

        # Assume labels are stored in a dictionary or follow a certain convention
        label = get_label_from_filename(video_filename)  # You should define this function based on your dataset

        return frames, label

# Define the dataset and dataloaders
train_dataset = KeypointDataset(npy_dir='data/splits/train', split='train')
val_dataset = KeypointDataset(npy_dir='data/splits/val', split='val')
test_dataset = KeypointDataset(npy_dir='data/splits/test', split='test')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
import torch.optim as optim
from sklearn.metrics import accuracy_score

def train_model(model, train_loader, val_loader, epochs=10, lr=1e-3):
    # Define optimizer and loss function
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()  # Assuming classification task

    best_val_acc = 0

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        train_acc = 0
        train_samples = 0

        # Training loop
        for data, label in train_loader:
            optimizer.zero_grad()

            # Forward pass
            output = model(data)

            # Compute loss and backpropagate
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()

            # Calculate accuracy
            _, predicted = torch.max(output, 1)
            train_acc += (predicted == label).sum().item()
            train_samples += label.size(0)

            train_loss += loss.item()

        # Calculate training accuracy
        train_acc /= train_samples
        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")

        # Validation
        val_acc = validate_model(model, val_loader)
        print(f"Validation Accuracy: {val_acc:.4f}")

        # Save the best model based on validation accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_stgcn_model.pth')

def validate_model(model, val_loader):
    model.eval()
    val_acc = 0
    val_samples = 0

    with torch.no_grad():
        for data, label in val_loader:
            output = model(data)
            _, predicted = torch.max(output, 1)
            val_acc += (predicted == label).sum().item()
            val_samples += label.size(0)

    val_acc /= val_samples
    return val_acc



### BEFORE TRAINING.


DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(DEVICE)


# Start training
train_model(stgcn_model, train_loader, val_loader)


In [ ]:
def test_model(model, test_loader):
    model.eval()
    test_acc = 0
    test_samples = 0

    with torch.no_grad():
        for data, label in test_loader:
            output = model(data)
            _, predicted = torch.max(output, 1)
            test_acc += (predicted == label).sum().item()
            test_samples += label.size(0)

    test_acc /= test_samples
    print(f"Test Accuracy: {test_acc:.4f}")

# Test the model
test_model(stgcn_model, test_loader)
